In [10]:
from pydantic import BaseModel
from openai import OpenAI

class UserInfo(BaseModel):
    name: str
    age: int
    city: str

# 🔑 ルートを object にし、その中に配列フィールドを置く
class People(BaseModel):
    users: list[UserInfo]

client = OpenAI()

# プロンプト側で「users 配列で返して」と明示
messages = [
    {
        "role": "developer",
        "content": (
            "あなたは情報抽出アシスタントです。"
            "出力は必ず次の JSON 形式のみで返してください："
            '{"users":[{name:str, age:int, city:str}, …]}'
        ),
    },
    {
        "role": "user",
        "content": (
            "私の名前は田中太郎、30歳、東京在住です。"
            "私の名前は鈴木健太、28歳、大阪在住です。"
        ),
    },
]

response = client.responses.parse(
    model="gpt-4.1",
    input=messages,
    text_format=People      # ← ここは People (object) に変更
)

people: People = response.output_parsed
for p in people.users:
    print(f"{p.name} / {p.age} / {p.city}")


田中太郎 / 30 / 東京
鈴木健太 / 28 / 大阪
